In [1]:
# UPDATED - ONLY ONE TOPIC ASSIGNED TO EACH DOCUMENT
# creating a topic_pro_df and csv file containing a dataframe of 
#the topic proportions, tag and tag ID for each document
import gensim
from gensim import corpora
import sqlite3
import sys
import csv
import re
import itertools
import pandas as pd

In [2]:
model =  gensim.models.LdaModel.load('lda_model175')

In [3]:
# recreate the corpus
text_lemmas = []
con = sqlite3.connect('news.db')
cur = con.cursor()
for row in range(36162):
	cur.execute("SELECT Words FROM BBCnews WHERE ID = ?;", (row + 1,))
	rowText = cur.fetchone()[0]
	rowText = re.sub(r'\s', '', rowText).split(',')
	text_lemmas.append(rowText)
id2word = corpora.Dictionary(text_lemmas)
id2word.filter_extremes(no_below = 30)
id2word.filter_tokens(bad_ids=[id2word.token2id['v'], id2word.token2id['p'],id2word.token2id['x'], id2word.token2id['b'], id2word.token2id['c'], id2word.token2id['q'],id2word.token2id['e'], id2word.token2id['j'],id2word.token2id['r'], id2word.token2id['g'], id2word.token2id['l'], id2word.token2id['n'], id2word.token2id['f'], id2word.token2id['w'],id2word.token2id['h'], id2word.token2id['mp'], id2word.token2id['de'], id2word.token2id['ap'],id2word.token2id['el'], id2word.token2id['le'], id2word.token2id['1m'], 
                               id2word.token2id['9m'], id2word.token2id['ch'], id2word.token2id['ni'], id2word.token2id['du'], id2word.token2id['co'], id2word.token2id['id'],id2word.token2id['qc'],id2word.token2id['gp'], id2word.token2id['2m'],id2word.token2id['6m'],id2word.token2id['st'],id2word.token2id['jo'],id2word.token2id['lt'],id2word.token2id['fa'], id2word.token2id['fc'], id2word.token2id['en'], id2word.token2id['4m'],id2word.token2id['hm'],id2word.token2id['3m'],id2word.token2id['a9'],id2word.token2id['na'],
                               id2word.token2id['3d'],id2word.token2id['vw'],id2word.token2id['5m'],id2word.token2id['m4'],id2word.token2id['vi'],id2word.token2id['al'],id2word.token2id['ok'],id2word.token2id['8m'],id2word.token2id['be'],id2word.token2id['da'],id2word.token2id['li'],id2word.token2id['ft'],id2word.token2id['sq'], id2word.token2id['ii'], id2word.token2id['fr'], id2word.token2id['bt'],id2word.token2id['ad'],id2word.token2id['jr'],id2word.token2id['ed'],id2word.token2id['g7'],id2word.token2id['fm'],id2word.token2id['rt'],id2word.token2id['pr'],id2word.token2id['oh'],id2word.token2id['km'],id2word.token2id['jp'],id2word.token2id['hq'],id2word.token2id['ie'],id2word.token2id['xi'],id2word.token2id['7m'],id2word.token2id['ho'],id2word.token2id['m6'],id2word.token2id['ai'],id2word.token2id['m1'],id2word.token2id['bc'],id2word.token2id['a1'],
                               id2word.token2id['o2'],id2word.token2id['1p'],id2word.token2id['mr'], id2word.token2id['would'], id2word.token2id['also'], id2word.token2id['go'],id2word.token2id['tell'], id2word.token2id['year'],id2word.token2id['could'], id2word.token2id['get'],id2word.token2id['come'], id2word.token2id['make'], id2word.token2id['take'], id2word.token2id['give'], id2word.token2id['know'], id2word.token2id['like'],id2word.token2id['way'], id2word.token2id['ms'],
                              id2word.token2id['people'], id2word.token2id['think'], id2word.token2id['time'],id2word.token2id['use'], id2word.token2id['see'], id2word.token2id['many'],id2word.token2id['want'], id2word.token2id['look'],id2word.token2id['even'], id2word.token2id['much'], id2word.token2id['back'],id2word.token2id['really'], id2word.token2id['still'],id2word.token2id['tens'], id2word.token2id['hundreds'], id2word.token2id['thousands'], id2word.token2id['mrs'], id2word.token2id['raf'], id2word.token2id['gwent'],id2word.token2id['dr'], id2word.token2id['bp']])
id2word.compactify()
corpus = [id2word.doc2bow(text) for text in text_lemmas]

In [4]:
# create a list of the tags from the sqlite database
tags = []
con = sqlite3.connect('news.db')
cur = con.cursor()
cur.execute("SELECT Tag FROM BBCnews;")
for i in cur:
    tags.append((i[0]))

In [13]:
# creating a dataframe with weight, topic ID and tag for all documents
# loop through topics for each document, only adding the first topic ID and weight 
cols = ['Weight','Topic','Tag']
unitop4doc_df = pd.DataFrame(columns = cols)
for i in range(36162):
    topicID = 'T'+ str(model.get_document_topics(corpus[i])[0][0])
    unitop4doc_df = unitop4doc_df.append([{cols[0]:model.get_document_topics(corpus[i])[0][1], cols[1]:topicID, cols[2]:tags[i]}],ignore_index=True)

In [14]:
unitop4doc_df

,Weight,Topic,Tag
0,0.035644,T11,Australia
1,0.020750,T13,Blogs
2,0.017634,T2,Scotland
3,0.139323,T0,Asia
4,0.027612,T14,Business
5,0.055039,T8,Berkshire
6,0.061329,T4,Birmingham
7,0.223957,T0,Kent
8,0.056124,T14,United Kingdom
9,0.099429,T3,Science


In [15]:
# save the dataframe as a csv file
unitop4doc_df.to_csv('unitop4doc_df.csv')

In [19]:
# updated list of uk cities for fusion table
uk_cities = ['Beds Bucks Herts', 'Bradford West Yorkshire', 'Cambridgeshire',
                   'Cornwall', 'Coventry Warwickshire', 'Cumbria', 'Derbyshire', 'Devon', 'Dorset', 'Essex', 'Gloucestershire', 
                   'Hampshire', 'Humber', 'Lancashire', 'Leeds', 'Leicestershire', 'Lincolnshire', 
                   'London', 'Manchester', 'Merseyside', 'Northamptonshire', 'Nottinghamshire', 'Oxfordshire', 
                   'Shropshire', 'South Yorkshire', 'Stoke Staffordshire', 'Surrey', 
                   'Tyne', 'Wiltshire', 'York North Yorkshire', 'Mid Wales', 'North East Wales', 'North West Wales', 'South East Wales', 'South West Wales', 
            'Edinburgh East Fife', 'Glasgow West', 'Highlands Islands', 'North East Orkney Shetland', 'Tayside Central']
uk_unitop4doc_df = pd.DataFrame(columns = cols)
uk_unitop4doc_df = unitop4doc_df[unitop4doc_df['Tag'].isin(uk_cities)]

In [20]:
# save the dataframe as a csv file
uk_unitop4doc_df.to_csv('uk_unitop4doc_df.csv')